In [15]:
import pandas as pd
import pprint as pp
import json
import os


In [2]:
# get team IDs

from nba_api.stats.static import teams

nba_teams = teams.get_teams()
team_df = pd.DataFrame(nba_teams)
# print(team_df)

print(f'teams variables: {list(team_df)}')
print()

team_ids = []
for i in team_df.index:
#     print(team_df.iloc[i]['id'])
    team_ids.append(team_df.iloc[i]['id'])
    
print(team_ids)


teams variables: ['id', 'full_name', 'abbreviation', 'nickname', 'city', 'state', 'year_founded']

[1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


In [3]:
# create df with team id, name, and location
team_df = team_df.loc[:,['id','full_name','city']]
team_df = team_df.rename(columns={'id':'Team ID','full_name':'Team Name','city':'City'})
team_df.head()

,Team ID,Team Name,City
0,1610612737,Atlanta Hawks,Atlanta
1,1610612738,Boston Celtics,Boston
2,1610612739,Cleveland Cavaliers,Cleveland
3,1610612740,New Orleans Pelicans,New Orleans
4,1610612741,Chicago Bulls,Chicago


In [4]:
# get players by team roster for 2021-22
# api library
from nba_api.stats.endpoints import commonteamroster

# initalize lists for constructing dictionary
t_id = []
player_name = []
height = []
weight = []
birthday = []
age = []
school = []
player_id = []
failed_ids = []

# function to request team roster data
def roster_api(id):
    # api call
        data = commonteamroster.CommonTeamRoster(team_id=id,season='2021-22')

        # convert to dictonary format
        data_dict = data.get_dict()

        # store results
        results = data_dict['resultSets'][0]

        # store team roster
        roster = results['rowSet']
        print()

        # iterate through roster to save player info in lists by index number
        for player in roster:

            t_id.append(player[0])
            player_name.append(player[3])
            height.append(player[8])
            weight.append(player[9])
            birthday.append(player[10])
            age.append(player[11])
            school.append(player[13])
            player_id.append(player[14])

        print(f'download for {id} successful')
        print('*****************************')
        print()


# iterate through team ids to download rosters for 2021-22
for id in team_ids:
    print(f'processing data for team id: {id}')
    
    # if first two requests for data fails, store list of failed team_IDs
    try:
        roster_api(id)
    except Exception as e:
        print(e)
        try:
            roster_api(id)
        except Exception as e:
            print(e)
            failed_ids.append(id)
            pass
        
print(failed_ids)



processing data for team id: 1610612737

download for 1610612737 successful
*****************************

processing data for team id: 1610612738

download for 1610612738 successful
*****************************

processing data for team id: 1610612739

download for 1610612739 successful
*****************************

processing data for team id: 1610612740

download for 1610612740 successful
*****************************

processing data for team id: 1610612741

download for 1610612741 successful
*****************************

processing data for team id: 1610612742

download for 1610612742 successful
*****************************

processing data for team id: 1610612743

download for 1610612743 successful
*****************************

processing data for team id: 1610612744

download for 1610612744 successful
*****************************

processing data for team id: 1610612745

download for 1610612745 successful
*****************************

processing data for team id: 16106127

In [5]:
# iterate over failed APIs for missing data
for id in failed_ids:
    print(f'processing data for team id: {id}')
    
    # if first two requests for data fails, store list of failed team_IDs
    try:
        roster_api(id)
    except Exception as e:
        print(e)
        try:
            roster_api(id)
        except Exception as e:
            print(e)
            failed_again_ids.append(id)
            pass

In [6]:
# create dictionary
player_dict = {
    'Player ID': player_id,
    'Player Name': player_name,
    'Height': height,
    'Weight': weight,
    'Birthday': birthday,
    'Age': age,
    'School': school,
    'Team ID': t_id
}

# convert dictionary to dataframe
player_df = pd.DataFrame(player_dict)

# show results
player_df

,Player ID,Player Name,Height,Weight,Birthday,Age,School,Team ID
0,1626153,Delon Wright,6-5,185,"APR 26, 1992",30.0,Utah,1610612737
1,1630552,Jalen Johnson,6-8,219,"DEC 18, 2001",20.0,Duke,1610612737
2,1630536,Sharife Cooper,6-1,176,"JUN 11, 2001",21.0,Auburn,1610612737
3,1628989,Kevin Huerter,6-7,198,"AUG 27, 1998",23.0,Maryland,1610612737
4,1630219,Skylar Mays,6-4,205,"SEP 05, 1997",24.0,Louisiana State,1610612737
...,...,...,...,...,...,...,...,...
501,1630550,JT Thor,6-9,203,"AUG 26, 2002",19.0,Auburn,1610612766
502,1630539,Kai Jones,6-10,221,"JAN 19, 2001",21.0,Texas,1610612766
503,203486,Mason Plumlee,6-11,254,"MAR 05, 1990",32.0,Duke,1610612766
504,1629023,P.J. Washington,6-7,230,"AUG 23, 1998",23.0,Kentucky,1610612766


In [7]:
# merge tables
teams_players_df = pd.merge(player_df,team_df,how='left',on='Team ID')
teams_players_df.head()

,Player ID,Player Name,Height,Weight,Birthday,Age,School,Team ID,Team Name,City
0,1626153,Delon Wright,6-5,185,"APR 26, 1992",30.0,Utah,1610612737,Atlanta Hawks,Atlanta
1,1630552,Jalen Johnson,6-8,219,"DEC 18, 2001",20.0,Duke,1610612737,Atlanta Hawks,Atlanta
2,1630536,Sharife Cooper,6-1,176,"JUN 11, 2001",21.0,Auburn,1610612737,Atlanta Hawks,Atlanta
3,1628989,Kevin Huerter,6-7,198,"AUG 27, 1998",23.0,Maryland,1610612737,Atlanta Hawks,Atlanta
4,1630219,Skylar Mays,6-4,205,"SEP 05, 1997",24.0,Louisiana State,1610612737,Atlanta Hawks,Atlanta


In [8]:
# determine birth month
b_month = teams_players_df['Birthday'].str.slice(0,4)
teams_players_df['Birth Month'] = b_month

teams_players_df.head()

,Player ID,Player Name,Height,Weight,Birthday,Age,School,Team ID,Team Name,City,Birth Month
0,1626153,Delon Wright,6-5,185,"APR 26, 1992",30.0,Utah,1610612737,Atlanta Hawks,Atlanta,APR
1,1630552,Jalen Johnson,6-8,219,"DEC 18, 2001",20.0,Duke,1610612737,Atlanta Hawks,Atlanta,DEC
2,1630536,Sharife Cooper,6-1,176,"JUN 11, 2001",21.0,Auburn,1610612737,Atlanta Hawks,Atlanta,JUN
3,1628989,Kevin Huerter,6-7,198,"AUG 27, 1998",23.0,Maryland,1610612737,Atlanta Hawks,Atlanta,AUG
4,1630219,Skylar Mays,6-4,205,"SEP 05, 1997",24.0,Louisiana State,1610612737,Atlanta Hawks,Atlanta,SEP


In [14]:
# send file to csv
teams_players_df.to_csv('output/teams_players_2021_2022.csv', index=False)

In [52]:
# import player stat data
file = os.path.join('..','salaries','nba_stats.csv')
stats_df = pd.read_csv(file)
stats_df.head(10)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional
0,1,Precious Achiuwa,C,22,TOR,73,1725,12.7,0.503,0.259,...,0.4,2.1,2.5,0.070,NaN,-2.0,-0.6,-2.6,-0.2,achiupr01
1,2,Steven Adams,C,28,MEM,76,1999,17.6,0.560,0.003,...,3.8,3.0,6.8,0.163,NaN,1.0,1.0,2.0,2.0,adamsst01
2,3,Bam Adebayo,C,24,MIA,56,1825,21.8,0.608,0.008,...,3.6,3.5,7.2,0.188,NaN,1.7,2.1,3.8,2.7,adebaba01
3,4,Santi Aldama,PF,21,MEM,32,360,10.2,0.452,0.364,...,-0.1,0.4,0.3,0.044,NaN,-4.2,-1.5,-5.7,-0.3,aldamsa01
4,5,LaMarcus Aldridge,C,36,BRK,47,1050,19.6,0.604,0.100,...,2.1,1.0,3.1,0.141,NaN,1.3,-0.6,0.7,0.7,aldrila01
5,6,Nickeil Alexander-Walker,SG,23,TOT,65,1466,10.5,0.475,0.497,...,-1.1,1.1,0.1,0.003,NaN,-1.8,-1.1,-2.9,-0.3,alexani01
6,6,Nickeil Alexander-Walker,SG,23,NOP,50,1317,10.5,0.474,0.483,...,-1.1,0.9,-0.1,-0.005,NaN,-1.7,-1.3,-3.0,-0.3,alexani01
7,6,Nickeil Alexander-Walker,SG,23,UTA,15,149,10.2,0.497,0.688,...,0.0,0.2,0.2,0.070,NaN,-2.9,1.2,-1.7,0.0,alexani01
8,7,Grayson Allen,SG,26,MIL,66,1805,12.7,0.609,0.684,...,2.8,1.4,4.2,0.110,NaN,0.6,-0.2,0.4,1.1,allengr01
9,8,Jarrett Allen,C,23,CLE,56,1809,23.0,0.698,0.018,...,5.4,3.0,8.5,0.225,NaN,2.7,1.2,3.9,2.7,allenja01


In [60]:
# select total stat for players with duplicate entries
duplicate_df = stats_df.loc[stats_df['Tm'] == 'TOT']
duplicate_df.head()

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional
5,6,Nickeil Alexander-Walker,SG,23,TOT,65,1466,10.5,0.475,0.497,...,-1.1,1.1,0.1,0.003,NaN,-1.8,-1.1,-2.9,-0.3,alexani01
11,10,Justin Anderson,SF,28,TOT,16,316,10.4,0.493,0.621,...,0.2,0.2,0.4,0.054,NaN,-1.9,-1.3,-3.2,-0.1,anderju01
22,19,D.J. Augustin,PG,34,TOT,55,883,9.7,0.610,0.739,...,0.7,0.1,0.8,0.043,NaN,-1.2,-2.4,-3.6,-0.4,augusdj01
29,24,Marvin Bagley III,PF,22,TOT,48,1146,15.7,0.551,0.217,...,1.2,0.9,2.0,0.085,NaN,-0.5,-1.8,-2.3,-0.1,baglema01
51,44,DeAndre' Bembry,SF,27,TOT,56,1026,11.9,0.601,0.163,...,1.1,1.1,2.2,0.104,NaN,-2.2,1.5,-0.7,0.3,bembrde01


In [61]:
# create list of players' names with duplicated rows
duplicate_names = first_duplicate_df['Player']
print(duplicate_names)
len(duplicate_names)

5      Nickeil Alexander-Walker
11              Justin Anderson
22                D.J. Augustin
29            Marvin Bagley III
51              DeAndre' Bembry
                 ...           
768              Tremont Waters
776               Derrick White
793             Justise Winslow
801                Moses Wright
805              Thaddeus Young
Name: Player, Length: 97, dtype: object


97

In [58]:
# remove duplicated rows
single_df = stats_df[~stats_df.Player.isin(duplicate_names)]
single_df.head(10)

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player-additional
0,1,Precious Achiuwa,C,22,TOR,73,1725,12.7,0.503,0.259,...,0.4,2.1,2.5,0.070,NaN,-2.0,-0.6,-2.6,-0.2,achiupr01
1,2,Steven Adams,C,28,MEM,76,1999,17.6,0.560,0.003,...,3.8,3.0,6.8,0.163,NaN,1.0,1.0,2.0,2.0,adamsst01
2,3,Bam Adebayo,C,24,MIA,56,1825,21.8,0.608,0.008,...,3.6,3.5,7.2,0.188,NaN,1.7,2.1,3.8,2.7,adebaba01
3,4,Santi Aldama,PF,21,MEM,32,360,10.2,0.452,0.364,...,-0.1,0.4,0.3,0.044,NaN,-4.2,-1.5,-5.7,-0.3,aldamsa01
4,5,LaMarcus Aldridge,C,36,BRK,47,1050,19.6,0.604,0.100,...,2.1,1.0,3.1,0.141,NaN,1.3,-0.6,0.7,0.7,aldrila01
8,7,Grayson Allen,SG,26,MIL,66,1805,12.7,0.609,0.684,...,2.8,1.4,4.2,0.110,NaN,0.6,-0.2,0.4,1.1,allengr01
9,8,Jarrett Allen,C,23,CLE,56,1809,23.0,0.698,0.018,...,5.4,3.0,8.5,0.225,NaN,2.7,1.2,3.9,2.7,allenja01
10,9,Jose Alvarado,PG,23,NOP,54,834,16.4,0.520,0.374,...,1.0,1.1,2.1,0.121,NaN,-0.5,2.4,1.9,0.8,alvarjo01
14,11,Kyle Anderson,PF,28,MEM,69,1484,14.4,0.506,0.232,...,1.0,2.5,3.5,0.113,NaN,-0.9,2.1,1.2,1.2,anderky01
15,12,Giannis Antetokounmpo,PF,27,MIL,67,2204,32.1,0.633,0.194,...,9.2,3.7,12.9,0.281,NaN,7.6,3.5,11.2,7.4,antetgi01


In [70]:
# append, sort, and select columns to show PER for each unique player
dfs = [single_df, duplicate_df]
clean_df = pd.concat(dfs).sort_values(by=['Rk'])
clean_df = clean_df[['Player', 'PER']]
print(len(clean_df))
clean_df.head(10)

605


,Player,PER
0,Precious Achiuwa,12.7
1,Steven Adams,17.6
2,Bam Adebayo,21.8
3,Santi Aldama,10.2
4,LaMarcus Aldridge,19.6
5,Nickeil Alexander-Walker,10.5
8,Grayson Allen,12.7
9,Jarrett Allen,23.0
10,Jose Alvarado,16.4
11,Justin Anderson,10.4


In [82]:
# merge PER data with other player info
merge_df = pd.merge(
    clean_df,
    teams_players_df[['Player Name','Height','Weight','Age','School','Team Name','City','Birth Month']], 
    how='outer', 
    left_on='Player',
    right_on='Player Name')
merge_df

,Player,PER,Player Name,Height,Weight,Age,School,Team Name,City,Birth Month
0,Precious Achiuwa,12.7,Precious Achiuwa,6-8,225,22.0,Memphis,Toronto Raptors,Toronto,SEP
1,Steven Adams,17.6,Steven Adams,6-11,265,28.0,Pittsburgh,Memphis Grizzlies,Memphis,JUL
2,Bam Adebayo,21.8,Bam Adebayo,6-9,255,24.0,Kentucky,Miami Heat,Miami,JUL
3,Santi Aldama,10.2,Santi Aldama,6-11,215,21.0,Loyola-Maryland,Memphis Grizzlies,Memphis,JAN
4,LaMarcus Aldridge,19.6,LaMarcus Aldridge,6-11,250,36.0,Texas-Austin,Brooklyn Nets,Brooklyn,JUL
...,...,...,...,...,...,...,...,...,...,...
639,NaN,NaN,Juancho Hernangomez,6-9,214,26.0,Estudiantes,Utah Jazz,Utah,SEP
640,NaN,NaN,Bojan Bogdanovic,6-7,226,33.0,Fenerbahce,Utah Jazz,Utah,APR
641,NaN,NaN,Xavier Tillman,6-8,245,23.0,Michigan State,Memphis Grizzlies,Memphis,JAN
642,NaN,NaN,Kristaps Porzingis,7-3,240,26.0,Cajasol Sevilla,Washington Wizards,Washington,AUG


In [92]:
# match players if there is time later - use data from the same source if possible
missing_df =  merge_df.loc[(merge_df['Player'].isnull()) | (merge_df['Player Name'].isnull())]
missing_df.to_csv('output/missing.csv')
missing_df

,Player,PER,Player Name,Height,Weight,Age,School,Team Name,City,Birth Month
9,Justin Anderson,10.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,OG Anunoby,14.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,Trevor Ariza,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Joel Ayayi,3.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,Cat Barber,-11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
639,NaN,NaN,Juancho Hernangomez,6-9,214,26.0,Estudiantes,Utah Jazz,Utah,SEP
640,NaN,NaN,Bojan Bogdanovic,6-7,226,33.0,Fenerbahce,Utah Jazz,Utah,APR
641,NaN,NaN,Xavier Tillman,6-8,245,23.0,Michigan State,Memphis Grizzlies,Memphis,JAN
642,NaN,NaN,Kristaps Porzingis,7-3,240,26.0,Cajasol Sevilla,Washington Wizards,Washington,AUG


In [89]:
# filter out rows with missing data
final_df = merge_df.loc[(merge_df['Player'].notnull()) & (merge_df['Player Name'].notnull())]
final_df.to_csv('output/clean_data.csv')
final_df

,Player,PER,Player Name,Height,Weight,Age,School,Team Name,City,Birth Month
0,Precious Achiuwa,12.7,Precious Achiuwa,6-8,225,22.0,Memphis,Toronto Raptors,Toronto,SEP
1,Steven Adams,17.6,Steven Adams,6-11,265,28.0,Pittsburgh,Memphis Grizzlies,Memphis,JUL
2,Bam Adebayo,21.8,Bam Adebayo,6-9,255,24.0,Kentucky,Miami Heat,Miami,JUL
3,Santi Aldama,10.2,Santi Aldama,6-11,215,21.0,Loyola-Maryland,Memphis Grizzlies,Memphis,JAN
4,LaMarcus Aldridge,19.6,LaMarcus Aldridge,6-11,250,36.0,Texas-Austin,Brooklyn Nets,Brooklyn,JUL
...,...,...,...,...,...,...,...,...,...,...
599,Gabe York,12.7,Gabe York,6-3,190,28.0,Arizona,Indiana Pacers,Indiana,AUG
600,Thaddeus Young,17.0,Thaddeus Young,6-8,235,34.0,Georgia Tech,Toronto Raptors,Toronto,JUN
601,Trae Young,25.4,Trae Young,6-1,164,23.0,Oklahoma,Atlanta Hawks,Atlanta,SEP
602,Omer Yurtseven,17.4,Omer Yurtseven,6-11,275,24.0,Georgetown,Miami Heat,Miami,JUN


In [93]:
# analysis
final_df.describe()

,PER,Age
count,467.000000,467.000000
mean,13.437259,26.092077
std,5.910163,4.333976
min,-45.200000,19.000000
25%,10.600000,23.000000
50%,13.300000,25.000000
75%,16.450000,29.000000
max,36.900000,42.000000


In [95]:
import matplotlib.pyplot as plt